<a href="https://colab.research.google.com/github/jaaaamj0711/kaggle_study/blob/master/Netflix/Netflix_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 💁 넷플릭스 영화 & 프로그램 추천

## 제목으로 추천 시스템 구현하기

In [126]:
data = pd.read_csv("/content/drive/MyDrive/netflix_titles.csv")

In [127]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [128]:
#removing stopwords
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
data['description'] = data['description'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(data['description'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(7787, 17905)

In [129]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [130]:
indices = pd.Series(data.index, index=data['title']).drop_duplicates()

In [131]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return data['title'].iloc[movie_indices]

In [132]:
get_recommendations('Black Mirror')

7088                                 Til Death Do Us Part
4683                                   Origins Collection
7191                             Transformers: Cyberverse
2979                                            Inception
277                                              Abby Sen
1532                                          Creeped Out
4686                                              Osmosis
562                                             Apollo 18
1078    Brave Blue World: Racing to Solve Our Water Cr...
2233                                     Forbidden Planet
Name: title, dtype: object

In [133]:
get_recommendations('Friends')

3904         Manhattan Romance
1025           BoJack Horseman
2015                  Episodes
5860                 Studio 54
1577                Dad's Army
7169         Trailer Park Boys
5629      Single Ladies Senior
4034                    Menahi
6549    The Larva Island Movie
7482                Warehoused
Name: title, dtype: object

추천해 주는 목록들을 봤을때 별로 정확해보이지는 않는다. 제목만 보고 추천을 해서 이러한 결과가 나왔다고 생각했다. 다른 변수들을 추가해서 다시 구현을 해보도록 하겠다.


## 제목, 배우, 감독, 주제, 설명으로 추천 시스템 구현하기

In [134]:
# 결측치 처리
all_data = data.fillna('')
all_data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [135]:
# 데이터 전처리 함수
def _clean(x):
        return str.lower(x.replace(" ", ""))

In [136]:
feature = ['title','director','cast','listed_in','description']
all_data = all_data[feature]

In [137]:
for features in feature:
    all_data[features] = all_data[features].apply(_clean)
    
all_data.head()

,title,director,cast,listed_in,description
0,3%,,"joãomiguel,biancacomparato,michelgomes,rodolfo...","internationaltvshows,tvdramas,tvsci-fi&fantasy",inafuturewheretheeliteinhabitanislandparadisef...
1,7:19,jorgemichelgrau,"demiánbichir,héctorbonilla,oscarserrano,azalia...","dramas,internationalmovies","afteradevastatingearthquakehitsmexicocity,trap..."
2,23:59,gilbertchan,"teddchan,stellachung,henleyhii,lawrencekoh,tom...","horrormovies,internationalmovies","whenanarmyrecruitisfounddead,hisfellowsoldiers..."
3,9,shaneacker,"elijahwood,johnc.reilly,jenniferconnelly,chris...","action&adventure,independentmovies,sci-fi&fantasy","inapostapocalypticworld,rag-dollrobotshideinfe..."
4,21,robertluketic,"jimsturgess,kevinspacey,katebosworth,aaronyoo,...",dramas,abrilliantgroupofstudentsbecomecard-countingex...


In [138]:
def create_soup(x):
    return x['title']+ ' ' + x['director'] + ' ' + x['cast'] + ' ' +x['listed_in']+' '+ x['description']

In [139]:
all_data['soup'] = all_data.apply(create_soup, axis=1)

In [140]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(all_data['soup'])

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [141]:
all_data = all_data.reset_index()
indices = pd.Series(all_data.index, index=all_data['title'])

In [142]:
def get_recommendations_new(title, cosine_sim=cosine_sim):
    title=title.replace(' ','').lower()
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return data['title'].iloc[movie_indices]

In [143]:
get_recommendations_new('Black Mirror', cosine_sim2)

1844                                       Dracula
6267           The Dark Crystal: Age of Resistance
7492                                Watership Down
2846    How to Live Mortgage Free with Sarah Beeny
7411                                         Vexed
350                       Ainsley Eats the Streets
359                                 Ajaibnya Cinta
398                                   Alien Worlds
449                              Amazing Interiors
2048                             Everyday Miracles
Name: title, dtype: object

In [144]:
get_recommendations_new('Friends', cosine_sim2)

2015                               Episodes
2263                                Frasier
6082                 The Andy Griffith Show
5818                             Still Game
1577                             Dad's Army
4811                    Pee-wee's Playhouse
7120                        Toast of London
1300                                 Cheers
6953    The Twilight Zone (Original Series)
3899                        Man with a Plan
Name: title, dtype: object